<a href="https://colab.research.google.com/github/imvickykumar999/Weather-AI-Agent/blob/main/Weather_Report.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import requests

def get_weather(city: str) -> dict:
    """Fetch weather from Open-Meteo (free, no key needed)."""
    try:
        # Step 1: Geocode city → lat/lon
        geo_resp = requests.get(
            "https://geocoding-api.open-meteo.com/v1/search",
            params={"name": city, "count": 1, "language": "en", "format": "json"},
            timeout=10,
        )
        geo_resp.raise_for_status()
        geo_data = geo_resp.json()

        if not geo_data.get("results"):
            return {"status": "error", "error_message": f"City '{city}' not found."}

        lat = geo_data["results"][0]["latitude"]
        lon = geo_data["results"][0]["longitude"]
        resolved_name = geo_data["results"][0]["name"]
        country = geo_data["results"][0].get("country", "")

        # Step 2: Fetch current weather
        weather_resp = requests.get(
            "https://api.open-meteo.com/v1/forecast",
            params={
                "latitude": lat,
                "longitude": lon,
                "current_weather": "true",
            },
            timeout=10,
        )
        weather_resp.raise_for_status()
        weather_data = weather_resp.json()

        current = weather_data.get("current_weather", {})
        if not current:
            return {"status": "error", "error_message": "No current weather available."}

        temp = current.get("temperature")
        wind = current.get("windspeed")

        report = (
            f"The weather in {resolved_name}, {country} is "
            f"{temp}°C with wind speed {wind} km/h."
        )
        return {"status": "success", "report": report}

    except Exception as e:
        return {"status": "error", "error_message": str(e)}

In [3]:
get_weather("Noida")

{'status': 'success',
 'report': 'The weather in Noida, India is 32.8°C with wind speed 8.9 km/h.'}

In [4]:
!pip install flask==3.0.3 requests==2.32.3 python-dotenv==1.0.1 groq>=0.9.0 litellm>=1.43.0 aiohttp>=3.10.5 google-adk

In [5]:
import os
from google.colab import userdata

os.environ["GROQ_API_KEY"] = userdata.get('GROQ_API_KEY')

In [6]:
try:
    import litellm
except Exception:
    litellm = None

# dir(litellm)

In [7]:
import warnings
import logging

warnings.filterwarnings("ignore")
logging.basicConfig(level=logging.ERROR)

In [8]:
try:
    print(get_weather("New York"))
    print(get_weather("Paris"))
except Exception as e:
    print(f"[WeatherAPI test] Skipped due to: {e}")

{'status': 'success', 'report': 'The weather in New York, United States is 19.1°C with wind speed 16.5 km/h.'}
{'status': 'success', 'report': 'The weather in Paris, France is 24.0°C with wind speed 7.4 km/h.'}


In [9]:
GROQ_PRIMARY = "groq/llama3-70b-8192"
GROQ_FALLBACKS = [
    "groq/llama3-8b-8192",
    "groq/mixtral-8x7b-32768",
]

# ADK/LiteLlm uses OpenAI-style params; keep outputs short to reduce TPM.
DEFAULT_MAX_TOKENS = 256

In [10]:
from google.adk.models.lite_llm import LiteLlm  # Multi-provider

def make_groq_model(model_name: str) -> LiteLlm:
    return LiteLlm(
        model=model_name,
        api_key=os.getenv("GROQ_API_KEY"),
        # Most OpenAI-compatible providers accept "max_tokens"
        max_tokens=DEFAULT_MAX_TOKENS,
        # you can also add temperature/top_p if you want:
        # temperature=0.2,
        # top_p=0.9,
        # LiteLLM will do its own retrying if configured globally; we handle retries below anyway.
    )

In [11]:
make_groq_model(GROQ_PRIMARY)

LiteLlm(model='groq/llama3-70b-8192', llm_client=<google.adk.models.lite_llm.LiteLLMClient object at 0x7e9fab8c2550>)

In [12]:
from google.adk.agents import Agent

weather_agent = Agent(
    name="weather_agent_v1",
    model=make_groq_model(GROQ_PRIMARY),
    description="Provides weather information for specific cities.",
    instruction=(
        "You are a helpful weather assistant. "
        "When the user asks for the weather in a specific city, "
        "use the 'get_weather' tool to find the information. "
        "If the tool returns an error, inform the user politely. "
        "If the tool is successful, present the weather report clearly. "
        "Keep responses concise."
    ),
    tools=[get_weather],
)

print(f"Agent '{weather_agent.name}' created using model '{GROQ_PRIMARY}'.")


Agent 'weather_agent_v1' created using model 'groq/llama3-70b-8192'.


In [13]:
from google.adk.sessions import InMemorySessionService

APP_NAME = "weather_tutorial_app"
USER_ID = "user_1"
SESSION_ID = "session_001"  # fixed for simplicity

session_service = InMemorySessionService()
session_service

In [14]:
async def create_session():
    session = await session_service.create_session(
        app_name=APP_NAME,
        user_id=USER_ID,
        session_id=SESSION_ID
    )
    print(f"Session created: App='{APP_NAME}', User='{USER_ID}', Session='{SESSION_ID}'")
    return session

create_session()

<coroutine object create_session at 0x7e9fa1978e40>

In [15]:
from google.adk.runners import Runner

runner = Runner(
    agent=weather_agent,
    app_name=APP_NAME,
    session_service=session_service
)

print(f"Runner created for agent '{runner.agent.name}'.")

Runner created for agent 'weather_agent_v1'.


In [16]:
import re, asyncio

RATE_LIMIT_PATTERN = re.compile(r"try again in ([\d.]+)s", re.IGNORECASE)

async def _backoff_sleep(err_msg: str, attempt: int):
    """
    Sleep using server-suggested delay if present, else exponential backoff.
    """
    m = RATE_LIMIT_PATTERN.search(err_msg or "")
    if m:
        delay = float(m.group(1))
    else:
        delay = min(2 ** attempt, 20)  # cap to avoid long stalls
    await asyncio.sleep(delay)

In [17]:
def _is_rate_limit_error(e: Exception) -> bool:
    if litellm and isinstance(e, getattr(litellm, "RateLimitError", tuple())):
        return True
    msg = f"{type(e).__name__}: {e}"
    return "rate limit" in msg.lower() or "rate_limit_exceeded" in msg.lower()

In [18]:
async def _swap_model_to_fallback(agent: Agent, used: set) -> bool:
    """
    Switch agent.model to the next fallback model not yet used.
    Returns True if swapped, False if none left.
    """
    for cand in GROQ_FALLBACKS:
        if cand not in used:
            agent.model = make_groq_model(cand)
            print(f"[Model Fallback] Switched to: {cand}")
            used.add(cand)
            return True
    return False

In [19]:
from google.genai import types

async def call_agent_async(query: str, runner: Runner, user_id: str, session_id: str):
    """
    Sends a query to the agent with robust rate-limit handling and model fallback.
    """
    print(f"\n>>> User Query: {query}")

    # Keep the prompt short to reduce token usage.
    content = types.Content(role="user", parts=[types.Part(text=query)])

    final_response_text = "Agent did not produce a final response."  # Default
    max_attempts = 5
    attempted_models = {GROQ_PRIMARY}
    attempt = 0

    while attempt < max_attempts:
        attempt += 1
        try:
            # IMPORTANT: Trim conversation history to reduce tokens.
            # If your ADK Runner uses entire session history by default,
            # pass only the new message. (Runner handles context internally.)
            async for event in runner.run_async(
                user_id=user_id,
                session_id=session_id,
                new_message=content
            ):
                if event.is_final_response():
                    if getattr(event, "content", None) and event.content.parts:
                        final_response_text = event.content.parts[0].text
                    elif getattr(event, "actions", None) and getattr(event.actions, "escalate", None):
                        final_response_text = f"Agent escalated: {event.error_message or 'No specific message.'}"
                    break

            print(f"<<< Agent Response: {final_response_text}")
            return  # success

        except Exception as e:
            msg = f"{type(e).__name__}: {e}"
            if _is_rate_limit_error(e):
                print(f"[RateLimit] Attempt {attempt}/{max_attempts}: {msg}")

                # 1) first: wait as suggested / backoff
                await _backoff_sleep(str(e), attempt)

                # 2) then: try fallback model if still failing on next loop
                # (swap only once per attempt to keep logic simple)
                if await _swap_model_to_fallback(runner.agent, attempted_models):
                    continue

                # If no fallback left, loop will retry with same model after backoff
                continue

            # Non-rate-limit error: raise or print friendly message and stop
            print(f"[Error] Unhandled exception: {msg}")
            break

    # If we reach here, retries exhausted
    print("<<< Agent Response: We hit provider limits repeatedly. Please try again shortly or upgrade your Groq tier.")


In [20]:
async def run_conversation():
    await create_session()
    try:
        while True:
            try:
                user_input = input("\n>>> Enter your query (or 'exit' to quit): ")
            except (EOFError, KeyboardInterrupt):
                print("\nEnding the conversation.")
                break

            if user_input.strip().lower() == "exit":
                print("Ending the conversation.")
                break

            await call_agent_async(
                user_input.strip(),
                runner=runner,
                user_id=USER_ID,
                session_id=SESSION_ID
            )
    finally:
        # Let any pending SSL/HTTP2 writes flush before the loop shuts down
        await asyncio.sleep(0.25)


In [21]:
async def main():
    # No blanket task-cancelling in Colab/Jupyter (causes recursion errors)
    await run_conversation()

In [22]:
if __name__ == "__main__":
    try:
        await main()  # use top-level await in Colab
    except Exception as e:
        print(f"An error occurred: {e}")

Session created: App='weather_tutorial_app', User='user_1', Session='session_001'

>>> Enter your query (or 'exit' to quit): hello, how are you?

>>> User Query: hello, how are you?
<<< Agent Response: Hello! I'm doing well, thank you for asking. I'm here to help you with any weather-related questions you might have. What city's weather would you like to know about?

>>> Enter your query (or 'exit' to quit): what is temperature of noida?

>>> User Query: what is temperature of noida?
<<< Agent Response: The weather in Noida, India is 32.8°C with wind speed 8.9 km/h.

>>> Enter your query (or 'exit' to quit): compare its wind speed with dubai

>>> User Query: compare its wind speed with dubai
<<< Agent Response: Comparing the wind speeds, I see that Dubai has a wind speed of 20.2 km/h, which is higher than Noida's wind speed of 8.9 km/h.

>>> Enter your query (or 'exit' to quit): with that of patna

>>> User Query: with that of patna
<<< Agent Response: Comparing the wind speeds, I see 